In [ ]:
import itertools

import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate

from utils import normals_to_rgb

In [ ]:
def v(t, s):
    """Parametric surface in 3-D:
    v(t, s) = (t + 1) i + (s) j + (s^2 - t^2 + 1) k
    """
    return np.c_[t + 1,
                 s,
                 s ** 2 - t ** 2 + 1]

def v_t(t, s):
    """First derivative of parametric surface, v, wrt t:
    v_t(t, s) = i  + (-2 t) k
    """
    return np.c_[np.ones_like(t),
                 np.zeros_like(t),
                 - 2 * t]

def v_s(t, s):
    """First derivative of parametric surface, v, wrt s:
    v_s(t, s) = j + (2 s) k
    """
    return np.c_[np.zeros_like(s),
                 np.ones_like(s),
                 2 * s]

def f(x, y, z):
    """Vector field that passes through V(t, s):
    """
    return np.c_[np.ones_like(x),
                 np.ones_like(y),
                 np.ones_like(z)]

In [ ]:
def dblquad(func, bbox, args=(), degree=9):
    """Return the integral of a given 2-D function, `f(x, y)`, solution
    of which is carried by using the Gauss-Legendre quadrature in 2-D.
    Parameters
    ----------
    func : callable
        Integrand function.
    bbox : list or tuple
        Integration domain [min(x), max(x), min(y), max(y)].
    args : tuple, optional
        Additional arguments for `func`.
    degree : int, optional
        Degree of the Gauss-Legendre quadrature.
    Returns
    -------
    float
        Integral of a given function.
    """
    from scipy.special import roots_legendre
    if not callable(func):
        raise ValueError('`func` must be callable')
    if not isinstance(bbox, (list, tuple, np.ndarray, )):
        raise ValueError('Integration domain must be iterable.')
    points, w = roots_legendre(degree)
    x_a, x_b, y_a, y_b = bbox
    x_scaler = (x_b - x_a) / 2
    y_scaler = (y_b - y_a) / 2
    x_scaled = x_scaler * (points + 1.) + x_a
    y_scaled = y_scaler * (points + 1.) + y_a
    X, Y = np.meshgrid(x_scaled, y_scaled)
    val = (x_scaler * w) @ func(X, Y, *args) @ (y_scaler * w)
    return val


def elementwise_dblquad(points, values, degree=9, interp_func=None, **kwargs):
    """Return the approximate value of the integral for sampled 2-D
    data by using the Gauss-Legendre quadrature in 2-D.
    Parameters
    ----------
    points : numpy.ndarray
        Data points of shape (n, 2), where n is the number of points.
    values : numpy.ndarray
        Sampled integrand function values of shape (n, ). If the data
        is sampled over a grid it could also be of shape (m, m), where
        m corresponds to the number of data points coordinates.
    degree : int, optional
        Degree of the Gauss-Legendre quadrature.
    interp_func : callable, optional
        Interpolation function. If not set radial basis function
        interpolation is used: `scipy.interpolate.RBFInterpolator`.
    kwargs : dict, optional
        Additional keyword arguments for the interpolation function.
    Returns
    -------
    float
        Approximation of the integral for givend 2-D data.
    """
    if not isinstance(values, (np.ndarray, )):
        raise Exception('`values` must be array-like.')
    try:
        bbox = [points[:, 0].min().item(), points[:, 0].max().item(),
                points[:, 1].min().item(), points[:, 1].max().item()]
    except TypeError:
        print('`points` must be a 2-column array.')
    if interp_func is None:
        func = interpolate.Rbf(points[:, 0], points[:, 1], values, **kwargs)
    else:
        func = interp_func(points, values, **kwargs)
    return dblquad(func, bbox, degree=degree)

# Analitically

In [ ]:
# integration domain
t_a, t_b = -2, 2
s_a, s_b = -2, 2

# scattered points
t = np.linspace(t_a, t_b, 51)
s = np.linspace(s_a, s_b, 51)
T, S = np.meshgrid(t, s)

# scattered parametric surface
V = v(T.ravel(), S.ravel())

# visualization
fig = plt.figure(figsize=(3, 3))
ax = plt.axes(projection ='3d')
ax.plot(*V.T, 'bo', ms=1, alpha=0.5)
ax.set(xlabel='t', ylabel='s', zlabel='v')
ax.view_init(20, 135)
fig.tight_layout()
plt.show()

In [ ]:
# compute vector field F across the surface v
F = f(x, y, z)

# visualization
show_idx = np.random.randint(0, F.shape[0], size=21)
fig = plt.figure(figsize=(3, 3))
ax = plt.axes(projection ='3d')
ax.plot(*V.T, 'bo', ms=1, alpha=0.5)
ax.quiver(*V[show_idx, :].T,
          *F[show_idx, :].T,
          color='red',
          length=1,
          label='F(v(t, s))')
ax.set(xlabel='t', ylabel='s', zlabel='v')
ax.view_init(20, 135)
fig.tight_layout()
plt.show()

In [ ]:
# compute unit normals
V_T = v_t(T.ravel(), S.ravel())
V_S = v_s(T.ravel(), S.ravel())
n = np.cross(V_T, V_S)

fig = plt.figure(figsize=(3, 3))
ax = plt.axes(projection ='3d')
ax.plot(*V.T, 'bo', ms=1, alpha=0.5)
ax.quiver(*V[show_idx, :].T,
          *F[show_idx, :].T,
          color='red',
          normalize=True,
          length=1,
          label='F(v(t, s))')
ax.quiver(*V[show_idx, :].T,
          *n[show_idx, :].T,
          color='black',
          normalize=True,
          length=1,
          label='n')
ax.set(xlabel='t', ylabel='s', zlabel='v')
ax.view_init(20, 135)
fig.tight_layout()
plt.show()

In [ ]:
# vector field F across the surface v
fig = plt.figure(figsize=(5, 5))
ax = plt.axes(projection ='3d')
cs = ax.scatter(*V.T, s=10, c=values, )
cbar = fig.colorbar(cs, shrink=0.35, pad=0.15)
cbar.ax.set_ylabel('F n')
ax.set(xlabel='t', ylabel='s', zlabel='v(t, s)')
ax.view_init(20, 135)
fig.tight_layout()
plt.show()

In [ ]:
# vector field F across the surface v
fig = plt.figure(figsize=(4, 3))
ax = plt.axes()
cs = ax.scatter(*points.T, c=values)
cbar = fig.colorbar(cs)
cbar.ax.set_ylabel('F n')
ax.set(xlabel='t', ylabel='s')
plt.show()

In [ ]:
points = np.c_[T.ravel(), S.ravel()]  # 2-D set of points
values = np.sum(F * n, axis=1)  # integrand = F(V(t, s)) · n(x, y, z)
elementwise_dblquad(points, values)

In [ ]:
I_exact = 16
I_exact

# NURBS

In [ ]:
# point cloud representation from scattered parametric surface
x, y, z = V.T

In [ ]:
# B-spline interpolation
interpolator = interpolate.SmoothBivariateSpline(x, y, z)
z_interp = interpolator(x, y, grid=False)

# visualization
fig = plt.figure(figsize=(3, 3))
ax = plt.axes(projection ='3d')
ax.plot(x, y, z_interp, 'bo', ms=1, alpha=0.5)
ax.set(xlabel='x', ylabel='y', zlabel='z')
ax.view_init(25, 125)
plt.show()

# error
fig = plt.figure(figsize=(4, 3))
ax = plt.axes()
cs = ax.scatter(x, y, c=np.abs(z_interp - z))
cbar = fig.colorbar(cs)
cbar.ax.set_ylabel('abs. error in z')
ax.set(xlabel='x', ylabel='y')
plt.show()

In [ ]:
dzdx_interp = interpolator(x, y, dx=1, dy=0, grid=False)
dzdy_interp = interpolator(x, y, dx=0, dy=1, grid=False)
n_interp = np.c_[-dzdx_interp,
                 -dzdy_interp,
                 np.ones_like(dzdy_interp)]
np.all(np.isclose(n, n_interp))

In [ ]:
points = np.c_[x, y]  # 2-D set of points
values = np.sum(F * n_interp, axis=1)  # integrand = F(V(t, s)) · n(x, y, z)
elementwise_dblquad(points, values)

In [ ]:
I_exact

In [ ]:
interpolator.integral(t_a, t_b, s_a, s_b)